In [1]:
import json
import pennylane as qml
import pennylane.numpy as np

In [6]:
def is_product(state, subsystem, wires):
    """Determines if a pure quantum state can be written as a product state between 
    a subsystem of wires and their compliment.

    Args:
        state (numpy.array): The quantum state of interest.
        subsystem (list(int)): The subsystem used to determine if the state is a product state.
        wires (list(int)): The wire/qubit labels for the state. Use these for creating a QNode if you wish!

    Returns:
        (str): "yes" if the state is a product state or "no" if it isn't.
    """
    result = False
    dev = qml.device("default.qubit", wires=len(wires))
    
    @qml.qnode(dev)
    def circuit(sub):
        qml.QubitStateVector(state, wires=wires)
        return qml.density_matrix(sub)
    d_matrix_a = circuit(sub=subsystem)
    d_matrix_b = circuit(sub=[x for x in wires if x not in subsystem])
    p_matrix = qml.math.dot(d_matrix_a, d_matrix_b)
    result = qml.math.allequal(state, p_matrix.flatten())

    return "yes" if result else "no"


In [7]:
# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    state, subsystem, wires = ins
    state = np.array(state)
    output = is_product(state, subsystem, wires)
    return output

def check(solution_output: str, expected_output: str) -> None:
    assert solution_output == expected_output


In [8]:
test_cases = [['[[0.707107, 0, 0, 0.707107], [0], [0, 1]]', 'no'], ['[[1, 0, 0, 0], [0], [0, 1]]', 'yes']]

In [9]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[0.707107, 0, 0, 0.707107], [0], [0, 1]]'...
Correct!
Running test case 1 with input '[[1, 0, 0, 0], [0], [0, 1]]'...
Correct!
